# Sequence to Sequence Learning with Neural Networks

- [PyTorch implementation](https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb)
- https://github.com/bentrevett/pytorch-seq2seq
- https://arxiv.org/abs/1409.3215
- ドイツ語から英語へ翻訳するタスク

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import time

In [3]:
SEED = 1234

# 実験の再現性を保つための方法
# https://pytorch.org/docs/stable/notes/randomness.html
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
# Tokenizerの設定
# !python -m spacy download en
# !python -m spacy download de
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [6]:
def tokenize_de(text):
    # 入力のドイツ語の語順を逆転することで訓練がしやすくなる
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [7]:
print(tokenize_de('Guten morgen!'))
print(tokenize_en('Good morning!'))

['!', 'morgen', 'Guten']
['Good', 'morning', '!']


In [9]:
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

In [18]:
# Datasetのダウンロード
# Multi30k dataset: 英語、ドイツ語、フランス語の30000文規模のパラレルコーパス
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

print(len(train_data.examples))
print(len(valid_data.examples))
print(len(test_data.examples))
print(vars(train_data.examples[0]))

29000
1014
1000
{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [17]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)
len(SRC.vocab), len(TRG.vocab)

(7855, 5893)

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [20]:
# Iteratorの構築
# PyTorchのDataLoaderみたいなもの？
# TorchTextのBacketIteratorを使うとサイズの異なる文章のミニバッチを自動でpaddingしてくれる
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)

### Seq2Seqモデルの構築
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/
- https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/
- https://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html
- http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
- http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/

In [21]:
class Encoder(nn.Module):

    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = (src_sent_len, batch_size)
        embedded = self.dropout(self.embedding(src))
        # embedded = (src_sent_len, batch_size, emb_dim)
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = (src_sent_len, batch_size, hid_dim * n_directions)
        # hidden = (n_layers * n_directions, batch_size, hid_dim)
        # cell = (n_layers * n_directions, batch_size, hid_dim)

        return hidden, cell

In [22]:
class Decoder(nn.Module):

    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # 1ステップ分の処理に相当
        # input = (batch_size)
        # hidden = (n_layers * n_directions, batch_size, hid_dim)
        # cell = (n_layers * n_directions, batch_size, hid_dim)
        input = input.unsqueeze(0)
        # input = (1, batch_size)
        embedded = self.dropout(self.embedding(input))
        # embedded = (1, batch_size, emb_dim)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = (sent_len, batch_size, hid_dim * n_directions)
        # hidden = (n_layers * n_directions, batch_size, hid_dim)
        # cell = (n_layers * n_directions, batch_size, hid_dim)
        prediction = self.out(output.squeeze(0))
        # prediction = (batch_size, output_dim)

        return prediction, hidden, cell

In [23]:
# teacher forcingが0.75の時は75%の確率でground truthの単語を次のステップの入力とする
# 25%の確率で1ステップ前に出力した単語を入力とする
class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, 'Hidden dimensions of encoder and decoder must be equal!'
        assert encoder.n_layers == decoder.n_layers, 'Encoder and decoder must have equal number of layers!'

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src = (src_sent_len, batch_size)
        # trg = (trg_sent_len, batch_size)
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # 出力を保存するTensor
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        # Encoderの最後のhidden stateをDecoderの初期状態とする
        # Decoderへの最初の入力は <sos>
        input = trg[0, :]
        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)

        return outputs

In [25]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

In [26]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [27]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,899,013 trainable parameters


In [29]:
optimizer = optim.Adam(model.parameters())

PAD_IDX = TRG.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [30]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        # trg = (trg_sent_len, batch_size)
        # output = (trg_sent_len, batch_size, output_dim)
        # 0列めは<sos>なのでlossの計算に入れない
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        # PyTorchのCrossEntropyLossのinputはsoftmax前の値を入れる
        # targetはラベルそのまま入れる
        # trg = ((trg_sent_len - 1) * batch_size)
        # output = ((trg_sent_len - 1) * batch_size, output_dim)
        loss = criterion(output, trg)
        loss.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [31]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # teacher forcingはOFF
            output = model(src, trg, 0)

            # trg = (trg_sent_len, batch_size)
            # output = (trg_sent_len, batch_size, output_dim)

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            # trg = (trg_sent_len - 1) * batch_size)
            # output = (trg_sent_len - 1) * batch_size, output_dim)
            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [32]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [33]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 27s
	Train Loss: 5.065 | Train PPL: 158.417
	Valid Loss: 4.947 | Valid PPL: 140.725
Epoch: 02 | Time: 0m 26s
	Train Loss: 4.525 | Train PPL:  92.324
	Valid Loss: 4.841 | Valid PPL: 126.537
Epoch: 03 | Time: 0m 26s
	Train Loss: 4.200 | Train PPL:  66.666
	Valid Loss: 4.580 | Valid PPL:  97.471
Epoch: 04 | Time: 0m 26s
	Train Loss: 3.967 | Train PPL:  52.850
	Valid Loss: 4.512 | Valid PPL:  91.070
Epoch: 05 | Time: 0m 26s
	Train Loss: 3.790 | Train PPL:  44.264
	Valid Loss: 4.406 | Valid PPL:  81.937
Epoch: 06 | Time: 0m 26s
	Train Loss: 3.641 | Train PPL:  38.136
	Valid Loss: 4.276 | Valid PPL:  71.973
Epoch: 07 | Time: 0m 26s
	Train Loss: 3.512 | Train PPL:  33.506
	Valid Loss: 4.130 | Valid PPL:  62.158
Epoch: 08 | Time: 0m 26s
	Train Loss: 3.366 | Train PPL:  28.963
	Valid Loss: 4.033 | Valid PPL:  56.431
Epoch: 09 | Time: 0m 26s
	Train Loss: 3.228 | Train PPL:  25.240
	Valid Loss: 3.931 | Valid PPL:  50.965
Epoch: 10 | Time: 0m 26s
	Train Loss: 3.140 | Train PPL

In [35]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.905 | Test PPL:  49.669 |
